# Data Cleaning

the data (specifically the measurement-specific data) weighs a couple hundred megas.  
this notebook aims to reduce that by removing many trash columns, without any feature engineering involved.

**do not run this nb, it is just for reference**

## initializing

In [1]:
import pandas as pd
from numpy import nan

### defs

In [2]:
def count_nulls(data):
    null_counts = data.isnull().sum()
    print(null_counts[null_counts > 0].sort_values(ascending=False))
    return

In [3]:
def remove_mostly_empty_cols(df):
    
    print('# of cols pre cleanup:', len(df.columns))
    
    for col in df.columns:
        if df[col].isna().sum() > 0.5*len(df):
            df.drop(columns=col, inplace=True)
    
    print('# of cols post cleanup:', len(df.columns))
    print(f'\ncols:\n{list(df.columns)}')
    
    return df

In [4]:
def filter_cols_by_str(name):
    cond = df.columns.str.contains(name)
    filt = list(df.columns[cond])
    return filt

In [5]:
def compare_similar_cols(col, other_col):

    diff_rows = len(df[abs(1-df[other_col]/df[col]) > .1])
    print(f'\n# of rows where {col} and {other_col} differ in more than 10%:')
    print(f'{diff_rows} rows')

    if diff_rows < 0.1*len(df):
        df.drop(columns=other_col, inplace=True)
        
    else:
        print(f'too many dissimilar values, unable to drop {other_col}') 

### upload

In [6]:
raw_df = pd.read_csv('data\\raw_data.csv', low_memory=False)
raw_df.rename(str.lower, axis='columns', inplace=True)
print(raw_df.shape)
raw_df.sample(10)

(864863, 74)


,cst_cnt,btl_cnt,sta_id,depth_id,depthm,t_degc,salnty,o2ml_l,stheta,o2sat,...,r_phaeo,r_pres,r_samp,dic1,dic2,ta1,ta2,ph2,ph1,dic quality comment
483300,19362,483301,100.0 060.0,19-7412JD-CT-340-0515-10000600-0800A-5,800,4.46,34.430,NaN,27.291,NaN,...,NaN,806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
642574,26326,642575,090.0 030.0,19-8911NM-HY-316-1107-09000300-0000B-7,0,18.35,33.657,5.62,24.159,104.6,...,0.06,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62138,2054,62139,072.0 060.0,19-5010ES-HY-288-1800-07200600-0259A-3,259,7.00,34.020,1.90,26.650,27.9,...,NaN,261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159894,6108,159895,086.7 045.0,19-5307ES-HY-199-1930-08670450-0030A-7,30,9.54,33.970,NaN,26.225,NaN,...,NaN,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
675545,27569,675546,081.8 046.9,19-9310NM-MX-295-1348-08180469-0329A-3,329,7.73,34.172,1.10,26.668,16.4,...,NaN,331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
672416,27461,672417,093.3 060.0,19-9308NM-MX-225-0927-09330600-0144A-3,144,8.99,33.887,2.88,26.251,44.2,...,0.08,145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54772,1775,54773,050.0 110.0,19-5009BD-HY-254-0330-05001100-0086A-3,86,11.09,32.880,5.86,25.111,93.7,...,NaN,86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
679465,27704,679466,080.0 060.0,19-9403JD-PR-094-1810-08000600-0400A-7,400,6.30,34.169,0.95,26.862,13.7,...,NaN,403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3272,107,3273,056.7 136.0,19-4904HO-HY-096-0042-05671360-0250A-7,250,7.35,33.817,3.06,26.442,45.3,...,NaN,251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93414,3114,93415,100.0 090.0,19-5107PT-HY-195-1724-10000900-0050A-7,50,14.98,33.496,NaN,24.819,NaN,...,NaN,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
raw_df.duplicated().sum() # sanity check

0

In [8]:
df = raw_df.copy()

## cleanup

### nulls

we drop every column that has 50% nulls or more

In [9]:
filtered_df = remove_mostly_empty_cols(df)

# of cols pre cleanup: 74
# of cols post cleanup: 36

cols:
['cst_cnt', 'btl_cnt', 'sta_id', 'depth_id', 'depthm', 't_degc', 'salnty', 'o2ml_l', 'stheta', 'o2sat', 'oxy_µmol/kg', 'recind', 't_prec', 's_prec', 'p_qual', 'chlqua', 'phaqua', 'po4q', 'sio3qu', 'no2q', 'no3q', 'nh3q', 'c14a1q', 'c14a2q', 'darkaq', 'meanaq', 'r_depth', 'r_temp', 'r_potemp', 'r_salinity', 'r_sigma', 'r_sva', 'r_dynht', 'r_o2', 'r_o2sat', 'r_pres']


In [10]:
df = filtered_df.copy()

### junk columns

we remove columns whose names include:
- 'id', 'rec' and 'btl' (measurement identifiers)
- 'q' (quality of measurement)
- 'prec' (how precise the measurements are)

In [11]:
cols_to_drop = eval('+'.join([f'filter_cols_by_str("{name}")' # ha! a use case for eval!
 for name in ['id', 'rec', 'btl', 'q', 'prec']]))

cols_to_drop

['sta_id',
 'depth_id',
 'recind',
 't_prec',
 's_prec',
 'btl_cnt',
 'p_qual',
 'chlqua',
 'phaqua',
 'po4q',
 'sio3qu',
 'no2q',
 'no3q',
 'nh3q',
 'c14a1q',
 'c14a2q',
 'darkaq',
 'meanaq',
 't_prec',
 's_prec']

In [12]:
no_junk_df = df.drop(columns = cols_to_drop)
sum([col not in no_junk_df.columns
     for col in cols_to_drop]) == len(cols_to_drop) # sanity check

True

In [13]:
df = no_junk_df.copy()

### similar columns

the data pairs a column with a 'reported' column, and the data it holds is about identical. we aim to retain only the reported columns.

In [14]:
df.columns

Index(['cst_cnt', 'depthm', 't_degc', 'salnty', 'o2ml_l', 'stheta', 'o2sat',
       'oxy_µmol/kg', 'r_depth', 'r_temp', 'r_potemp', 'r_salinity', 'r_sigma',
       'r_sva', 'r_dynht', 'r_o2', 'r_o2sat', 'r_pres'],
      dtype='object')

In [15]:
duped_cols = df.columns[df.columns.str.contains('r_')].drop('r_potemp r_sva r_dynht r_pres'.split())
duped_cols

Index(['r_depth', 'r_temp', 'r_salinity', 'r_sigma', 'r_o2', 'r_o2sat'], dtype='object')

In [16]:
col_pairs = {
    'r_depth':'depthm',
    'r_temp':'t_degc',
    'r_salinity':'salnty',
    'r_sigma':'stheta',
    'r_o2':'o2ml_l',
    'r_o2sat':'o2sat',
}

for col in duped_cols:
    compare_similar_cols(col,col_pairs[col])


# of rows where r_depth and depthm differ in more than 10%:
31 rows

# of rows where r_temp and t_degc differ in more than 10%:
0 rows

# of rows where r_salinity and salnty differ in more than 10%:
1 rows

# of rows where r_sigma and stheta differ in more than 10%:
2 rows

# of rows where r_o2 and o2ml_l differ in more than 10%:
34 rows

# of rows where r_o2sat and o2sat differ in more than 10%:
26 rows


### column renaming and reordering

In [17]:
df = df.rename(
    {'cst_cnt': 'cruise',
        'oxy_µmol/kg': 'o2_molal',
        'r_depth': 'depth',
        'r_temp': 'temp',
        'r_potemp': 'pot_temp',
        'r_salinity': 'sal',
        'r_sigma': 'pot_dens',
        'r_sva': 'sva',
        'r_dynht': 'dyn_ht',
        'r_o2': 'o2',
        'r_o2sat': 'o2_sat',
        'r_pres': 'pres'},
    axis='columns'
).loc[:, ['temp',
          'pot_temp',
          'sva',
          'o2',
          'o2_molal',
          'o2_sat',
          'depth',
          'pres',
          'dyn_ht',
          'pot_dens',
          'sal',
          'cruise']]

df.sample(10)

,temp,pot_temp,sva,o2,o2_molal,o2_sat,depth,pres,dyn_ht,pot_dens,sal,cruise
601044,17.17,17.17,411.2,5.72,249.52200,103.6,0.0,0,0.000,23.770,32.788,24585
213463,7.90,7.88,151.3,2.65,115.28860,39.8,187.0,188,0.380,26.540,34.050,8160
550959,5.45,5.40,105.9,NaN,NaN,NaN,600.0,604,1.090,27.070,34.295,22422
644218,14.51,14.50,312.5,5.88,256.23590,101.3,84.0,84,0.260,24.840,33.391,26388
145446,9.46,9.43,144.4,0.44,19.14031,6.9,294.0,296,0.670,26.650,34.490,5479
643446,12.78,12.77,305.5,6.33,275.82990,105.0,42.0,42,0.140,24.890,33.013,26360
806585,18.50,18.50,400.9,5.85,238.15500,101.8,1.0,1,0.004,23.886,33.349,32142
514132,17.90,17.90,370.7,5.67,247.23730,104.6,11.0,11,0.040,24.200,33.575,20888
85296,4.49,4.42,80.5,0.39,16.95355,5.4,900.0,907,1.290,27.360,34.520,2835
23542,5.59,5.54,102.8,0.53,23.04518,7.5,591.0,595,0.900,27.100,34.360,765


## output

In [18]:
df.shape

(864863, 12)

In [19]:
df.columns

Index(['temp', 'pot_temp', 'sva', 'o2', 'o2_molal', 'o2_sat', 'depth', 'pres',
       'dyn_ht', 'pot_dens', 'sal', 'cruise'],
      dtype='object')

In [20]:
df.to_csv('data\\clean_data.csv', index=False)